In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

conn = sqlite3.connect('example.db')


def rsq(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, conn)


In [ ]:
# SELECT TOP(10)
# 	c.CalendarQuarterName,
# 	c.MonthName,
# 	c.CalendarDayOfYear
# FROM dbo.Calendar c
# WHERE
# 	-- Create dates from component parts
# 	DATEFROMPARTS(c.CalendarYear, c.CalendarMonth, c.Day) >= '2018-06-01' 
# 	AND c.DayName = 'Tuesday'
# ORDER BY
# 	c.FiscalYear,
# 	c.FiscalDayOfYear ASC;

In [8]:
rsq("""SELECT 
	c.CalendarQuarterName,
	c.MonthName,
	c.CalendarDayOfYear
FROM 
	Calendar c
WHERE
	-- Create dates from component parts
	STRFTIME('%Y-%m-%d', c.CalendarYear || '-' || c.CalendarMonth || '-' || c.Day) >= '2018-06-01' 
	AND c.DayName = 'Tuesday'
ORDER BY
	c.FiscalYear,
	c.FiscalDayOfYear ASC
LIMIT 10;
""")

,CalendarQuarterName,MonthName,CalendarDayOfYear
0,Q4,October,289
1,Q4,October,296
2,Q4,October,303
3,Q4,November,317
4,Q4,November,324
5,Q4,November,331
6,Q4,December,345
7,Q4,December,352
8,Q4,December,359
9,Q4,October,288


In [ ]:
SELECT -- Mark the date and time the lunar module touched down -- Use 24-hour notation for hours, so e.g., 9 PM is 21 
DATETIME2FROMPARTS(1969, 07, 20, 20, 17, 00, 000, 0) AS TheEagleHasLanded, -- Mark the date and time the lunar module took back off -- Use 24-hour notation for hours, so e.g., 9 PM is 21 
DATETIMEFROMPARTS(1969, 07, 21, 18, 54, 00, 000) AS MoonDeparture;

In [9]:
rsq("""
SELECT -- Mark the date and time the lunar module touched down -- Use 24-hour notation for hours, so e.g., 9 PM is 21 
DATETIME('1969-07-20 20:17:00') AS TheEagleHasLanded, -- Mark the date and time the lunar module took back off -- Use 24-hour notation for hours, so e.g., 9 PM is 21 
DATETIME('1969-07-21 18:54:00') AS MoonDeparture;""")

,TheEagleHasLanded,MoonDeparture
0,1969-07-20 20:17:00,1969-07-21 18:54:00


In [ ]:
SELECT -- Fill in the millisecond PRIOR TO chaos 
DATETIMEOFFSETFROMPARTS(2038, 01, 19, 03, 14, 07, 999, 0, 0, 3) AS LastMoment, -- Fill in the date and time when we will experience the Y2.038K problem -- Then convert to the Eastern Standard Time time zone 
DATETIMEOFFSETFROMPARTS(2038, 01, 19, 03, 14, 08, 0, 0, 0, 3) AT TIME ZONE 'Eastern Standard Time' AS TimeForChaos;

In [10]:
rsq("""SELECT 
DATETIME('2038-01-19 03:14:07.999', 'localtime') AS LastMoment,
-- Fill in the date and time when we will experience the Y2.038K problem -- Then convert to the Eastern Standard Time time zone 
DATETIME('2038-01-19 03:14:08.000', 'localtime', '-5 hours') AS TimeForChaos; -- -5 hours for Eastern Standard Time. Change accordingly for other timezones.""")

,LastMoment,TimeForChaos
0,2038-01-19 08:44:07,2038-01-19 03:44:08


In [19]:
rsq("""SELECT d.DateText AS String, -- Cast as DATE 
CAST(d.DateText AS DATE) AS StringAsDate, -- Cast as DATETIME2(7) 
CAST(d.DateText AS DATETIME2(7)) AS StringAsDateTime2 FROM Dates1 d;""")

,String,StringAsDate,StringAsDateTime2
0,2019-04-01 18:08:19.290,2019,2019
1,2019-04-07 06:14:30,2019,2019
2,2017-03-01,2017,2017
3,"April 4, 2016",0,0
4,2017-06-14 09:30:00.0068848,2017,2017
5,19970614,19970614,19970614
6,2004-09-14T21:03:06Z,2004,2004


In [ ]:
SET LANGUAGE 'GERMAN'

SELECT d.DateText AS String, -- Convert to DATE CONVERT(DATE, d.DateText) AS StringAsDate, -- Convert to DATETIME2(7) CONVERT(DATETIME2(7), d.DateText) AS StringAsDateTime2 FROM dbo.Dates d;

In [23]:
rsq("""
SELECT d.DateText AS String, 
DATE(d.DateText) AS StringAsDate, 
DATETIME(d.DateText) AS StringAsDateTime 
FROM Dates1 d;""")

,String,StringAsDate,StringAsDateTime
0,2019-04-01 18:08:19.290,2019-04-01,2019-04-01 18:08:19
1,2019-04-07 06:14:30,2019-04-07,2019-04-07 06:14:30
2,2017-03-01,2017-03-01,2017-03-01 00:00:00
3,"April 4, 2016",None,None
4,2017-06-14 09:30:00.0068848,2017-06-14,2017-06-14 09:30:00
5,19970614,None,None
6,2004-09-14T21:03:06Z,2004-09-14,2004-09-14 21:03:06


In [ ]:
SELECT d.DateText AS String, -- Parse as DATE using German 
PARSE(d.DateText AS DATE USING 'de-de') AS StringAsDate, -- Parse as DATETIME2(7) using German 
PARSE(d.DateText AS DATETIME2(7) USING 'de-de') AS StringAsDateTime2 FROM dbo.Dates d;

In [25]:
rsq("""
SELECT DateText AS String, 
       strftime('%Y-%m-%d', DateText) AS StringAsDate, 
       datetime(DateText) AS StringAsDateTime2 
FROM Dates1;""")

,String,StringAsDate,StringAsDateTime2
0,2019-04-01 18:08:19.290,2019-04-01,2019-04-01 18:08:19
1,2019-04-07 06:14:30,2019-04-07,2019-04-07 06:14:30
2,2017-03-01,2017-03-01,2017-03-01 00:00:00
3,"April 4, 2016",None,None
4,2017-06-14 09:30:00.0068848,2017-06-14,2017-06-14 09:30:00
5,19970614,None,None
6,2004-09-14T21:03:06Z,2004-09-14,2004-09-14 21:03:06


In [ ]:
DECLARE @OlympicsUTC NVARCHAR(50) = N'2016-08-08 23:00:00';

SELECT -- Fill in the time zone for Brasilia, Brazil 
SWITCHOFFSET(@OlympicsUTC, '-03:00') AS BrasiliaTime, -- Fill in the time zone for Chicago, Illinois 
SWITCHOFFSET(@OlympicsUTC, '-05:00') AS ChicagoTime, -- Fill in the time zone for New Delhi, India 
SWITCHOFFSET(@OlympicsUTC, '+05:30') AS NewDelhiTime;

In [30]:
rsq("""-- Brasilia, Brazil (UTC-3)
SELECT 
    datetime('2016-08-08T23:00:00Z', '-3 hours') AS BrasiliaTime, 
    -- Chicago, Illinois (UTC-5)
    datetime('2016-08-08T23:00:00Z', '-5 hours') AS ChicagoTime, 
    -- New Delhi, India (UTC+5:30)
    datetime('2016-08-08T23:00:00Z', '+5 hours 30 minutes') AS NewDelhiTime;""")

,BrasiliaTime,ChicagoTime,NewDelhiTime
0,2016-08-08 20:00:00,2016-08-08 18:00:00,None


In [ ]:
DECLARE @OlympicsClosingUTC DATETIME2(0) = '2016-08-21 23:00:00';

SELECT -- Fill in 7 hours back and a -07:00 offset TODATETIMEOFFSET(DATEADD(HOUR, -7, @OlympicsClosingUTC), '-07:00') AS PhoenixTime, -- Fill in 12 hours forward and a +12:00 offset.
TODATETIMEOFFSET(DATEADD(HOUR, 12, @OlympicsClosingUTC), '+12:00') AS TuvaluTime;

In [34]:
import datetime

# Define the parameter
OlympicsClosingUTC = datetime.datetime(2016, 8, 21, 23, 0, 0)
print(OlympicsClosingUTC)
rsq(f"""SELECT 
    -- Use strftime function to format the datetime output
    -- Use %Y-%m-%d %H:%M:%S.%f format for microseconds
    strftime('%Y-%m-%d %H:%M:%S.%f', 
        -- Use datetime function for adding hours and subtracting hours
        datetime('{OlympicsClosingUTC}', '-7 hours')) || '-07:00' AS PhoenixTime,
    strftime('%Y-%m-%d %H:%M:%S.%f', 
        datetime('{OlympicsClosingUTC}', '+12 hours')) || '+12:00' AS TuvaluTime;
""")

2016-08-21 23:00:00


,PhoenixTime,TuvaluTime
0,2016-08-21 16:00:00.00.000-07:00,2016-08-22 11:00:00.00.000+12:00


In [ ]:
DECLARE @OlympicsClosingUTC DATETIME2(0) = '2016-08-21 23:00:00';

SELECT -- Fill in 7 hours back and a -07:00 offset 
TODATETIMEOFFSET(DATEADD(HOUR, -7, @OlympicsClosingUTC), '-07:00') AS PhoenixTime, -- Fill in 12 hours forward and a +12:00 offset.
TODATETIMEOFFSET(DATEADD(HOUR, 12, @OlympicsClosingUTC), '+12:00') AS TuvaluTime;

In [37]:
rsq("""
SELECT 
    -- Fill in 7 hours back and a -07:00 offset 
    datetime('2016-08-21 23:00:00', '-7 hours') || '-07:00' AS PhoenixTime,
    -- Fill in 12 hours forward and a +12:00 offset.
    datetime('2016-08-21 23:00:00', '+12 hours') || '+12:00' AS TuvaluTime;""")


,PhoenixTime,TuvaluTime
0,2016-08-21 16:00:00-07:00,2016-08-22 11:00:00+12:00


In [ ]:
DECLARE
	@GoodDateINTL NVARCHAR(30) = '2019-03-01 18:23:27.920',
	@GoodDateDE NVARCHAR(30) = '13.4.2019',
	@GoodDateUS NVARCHAR(30) = '4/13/2019',
	@BadDate NVARCHAR(30) = N'SOME BAD DATE';

-- The prior solution using TRY_CAST
SELECT
	TRY_CAST(@GoodDateINTL AS DATETIME2(3)) AS GoodDateINTL,
	TRY_CAST(@GoodDateDE AS DATE) AS GoodDateDE,
	TRY_CAST(@GoodDateUS AS DATE) AS GoodDateUS,
	TRY_CAST(@BadDate AS DATETIME2(3)) AS BadDate;

SELECT TRY_PARSE(@GoodDateINTL AS DATETIME2(3)) AS GoodDateINTL, -- Fill in the correct region based on our input -- Be sure to match these data types with the -- TRY_CAST() examples above! 
TRY_PARSE(@GoodDateDE AS DATE USING 'de-de') AS GoodDateDE, TRY_PARSE(@GoodDateUS AS DATE USING 'en-us') AS GoodDateUS, -- TRY_PARSE can't fix completely invalid dates 
TRY_PARSE(@BadDate AS DATETIME2(3) USING 'sk-sk') AS BadDate;

In [40]:
rsq("""
-- Declare variables and set their values
WITH vars AS (
  SELECT 
    '2019-03-01 18:23:27.920' AS GoodDateINTL,
    '13.4.2019' AS GoodDateDE,
    '4/13/2019' AS GoodDateUS,
    'SOME BAD DATE' AS BadDate
)
-- The prior solution using TRY_CAST (SQLite equivalent is CAST)
SELECT
    CAST(v.GoodDateINTL AS DATETIME) AS GoodDateINTL,
    CAST(v.GoodDateDE AS DATE) AS GoodDateDE,
    CAST(v.GoodDateUS AS DATE) AS GoodDateUS,
    CAST(v.BadDate AS DATETIME) AS BadDate
FROM vars v;

""").to_sql('vars',conn,if_exists='replace',index=False)

1

In [4]:
rsq("""
-- The TRY_PARSE equivalent in SQLite is using the strftime function with format codes and modifiers.
-- Here's how to do it:
SELECT 
    strftime('%Y-%m-%d %H:%M:%f', '2019-03-01 18:23:27.920') AS GoodDateINTL,
    datetime('13.4.2019', '+1 day') AS GoodDateDE,
    date('4/13/2019', '+3 day') AS GoodDateUS,
    -- There's no TRY_PARSE equivalent in SQLite so we'll use a CASE statement to handle bad data
    CASE 
        WHEN strftime('%Y-%m-%d %H:%M:%f', 'SOME BAD DATE') IS NULL THEN 'Invalid date'
        ELSE strftime('%Y-%m-%d %H:%M:%f', 'SOME BAD DATE') 
    END AS BadDate; """)

,GoodDateINTL,GoodDateDE,GoodDateUS,BadDate
0,2019-03-01 18:23:27.920,None,None,Invalid date


In [ ]:
WITH EventDates AS
(
    SELECT
        -- Fill in the missing try-conversion function
        TRY_CONVERT(DATETIME2(3), it.EventDate) AT TIME ZONE it.TimeZone AS EventDateOffset,
        it.TimeZone
    FROM dbo.ImportedTime it
        INNER JOIN sys.time_zone_info tzi
			ON it.TimeZone = tzi.name
)
SELECT
    -- Fill in the approppriate event date to convert
	CONVERT(NVARCHAR(50), ed.EventDateOffset) AS EventDateOffsetString,
	CONVERT(DATETIME2(0), ed.EventDateOffset) AS EventDateLocal,
	ed.TimeZone,
    -- Convert from a DATETIMEOFFSET to DATETIME at UTC
	CAST(ed.EventDateOffset AT TIME ZONE 'UTC' AS DATETIME2(0)) AS EventDateUTC,
    -- Convert from a DATETIMEOFFSET to DATETIME with time zone
	CAST(ed.EventDateOffset AT TIME ZONE 'US Eastern Standard Time'  AS DATETIME2(0)) AS EventDateUSEast
FROM EventDates ed;

In [9]:
rsq("""WITH EventDates AS
(
    SELECT
        -- Fill in the missing date-time conversion function
        strftime('%Y-%m-%d %H:%M:%f', it.EventDate || ' ' || it.TimeZone, 'utc') AS EventDateOffset,
        it.TimeZone
    FROM ImportedTime it
        INNER JOIN time_zone_info tzi
			ON it.TimeZone = tzi.name
)
SELECT
    -- Fill in the appropriate event date to convert
	EventDateOffset AS EventDateOffsetString,
	datetime(EventDateOffset, 'localtime') AS EventDateLocal,
	ed.TimeZone,
    -- Convert from a DATETIMEOFFSET to DATETIME at UTC
	datetime(EventDateOffset, 'utc') AS EventDateUTC,
    -- Convert from a DATETIMEOFFSET to DATETIME with time zone
	datetime(EventDateOffset, 'US/Eastern') AS EventDateUSEast
    FROM EventDates ed;""")

DatabaseError: Execution failed on sql 'WITH EventDates AS
(
    SELECT
        -- Fill in the missing date-time conversion function
        strftime('%Y-%m-%d %H:%M:%f', it.EventDate || ' ' || it.TimeZone, 'utc') AS EventDateOffset,
        it.TimeZone
    FROM ImportedTime it
        CREATE TABLE time_zone_info (
            name TEXT NOT NULL,
            abbreviation TEXT NOT NULL,
            utc_offset INTEGER NOT NULL,
            is_dst INTEGER NOT NULL,
            PRIMARY KEY (name)
        );
			ON it.TimeZone = tzi.name
)
SELECT
    -- Fill in the appropriate event date to convert
	EventDateOffset AS EventDateOffsetString,
	datetime(EventDateOffset, 'localtime') AS EventDateLocal,
	ed.TimeZone,
    -- Convert from a DATETIMEOFFSET to DATETIME at UTC
	datetime(EventDateOffset, 'utc') AS EventDateUTC,
    -- Convert from a DATETIMEOFFSET to DATETIME with time zone
	datetime(EventDateOffset, 'US/Eastern') AS EventDateUSEast
    FROM EventDates ed;': near "CREATE": syntax error